# Open Fall Detector

## To do
Implement a classificator based on MobileNetV2 or another light CNN architecture to extract features from Multicam and URFall datasets. Feed the features to a LSTM network with visual attention mechanism to classify features in ADL, falling and lying down categories.


1. <del>Remove delay frames and split Multicam in one folder per class</del>
2. <del> Split URFall in one folder per class </del>
3. Use Pytorch EfficientNet to classify images
4. Use Pytorch EfficientNet just to extract features
5. Feed LSTM with features extracted by the CNN
6. Implement visual attention

In [3]:
import pandas as pd
import numpy as np
import os
import shutil
from natsort import natsorted
import glob
import cv2
from tqdm import tqdm

## Utilities

In [4]:
def listdir_nohidden_sorted(path):
    return natsorted(glob.glob(os.path.join(path, '*')))

def safe_mkdir(path):
    try:
        os.mkdir(path)
    except FileExistsError:
        pass
    
def extract_frames(video_path, frame_name, frame_format, dest):
    import cv2
    capture = cv2.VideoCapture(video_path)
    count = 1
    safe_mkdir(dest)

    while True:
        success, frame = capture.read()
        if not success:
            break

        safe_mkdir(dest)
        file = f'{dest}/{frame_name}{str(count).zfill(6)}.{frame_format}'
        if not os.path.exists(file):    
            cv2.imwrite(file, frame)
            print(f'extracting {file} to {dest} --- {count} frames moved.')
        else:
            print('file already exists.')
        count += 1

## Paths

In [ ]:
MULTICAM_PATH = './data/multicam'
MULTICAM_FRAMES_PATH = './data/multicam_frames'
MULTICAM_EXCEL_PATH = './data/multicam.xlsx'
MULTICAM_FRAMES_CLASSES_PATH = './data/multicam_frames_classes'

## 1: Preprocessing Multicam

#### Global Multicam Variables

In [21]:
img_name_format = f'./data/{MULTICAM_FRAMES_PATH}/chuteXX-camY-img'
start = len(f'./data/{MULTICAM_FRAMES_PATH}/')
chute_number_start = img_name_format.find('chute', start) + 6
chute_number_end = chute_number_start + 2
cam_number_index = img_name_format.find('cam', start) + 4
img_number_start = img_name_format.find('img', start) + 4 
img_number_end = img_number_start + 6

In [22]:
multicam_delay_frames = pd.read_excel(MULTICAM_EXCEL_PATH, sheet_name='delay_frames')
multicam_delay_frames.head()

,scenario_number,camera_1,camera_2,camera_3,camera_4,camera_5,camera_6,camera_7,camera_8
0,1,3,3,8,4,23,6,6,0
1,2,25,40,0,16,18,33,33,6
2,3,12,16,8,16,35,20,20,0
3,4,72,79,78,0,68,82,83,56
4,5,17,24,5,11,7,26,28,0


In [4]:
multicam_labels = pd.read_excel(MULTICAM_EXCEL_PATH, sheet_name='labels')
multicam_labels.head()

,scenario_number,camera_reference,period_start,period_end,class
0,1,11,874,1011,1
1,1,11,1012,1079,6
2,1,11,1080,1108,2
3,1,11,1109,1285,3
4,2,4,308,374,1


### 1.1 Extracting frames from videos

In [ ]:
safe_mkdir(MULTICAM_FRAMES_PATH)

chutes = listdir_nohidden_sorted(MULTICAM_PATH)
index = 1
for chute in tqdm(chutes):
    cams = listdir_nohidden_sorted(chute)
    
    for c, cam in enumerate(cams):
        file_path = cam
        number = str(index).zfill(2)
        dest = f'{MULTICAM_FRAMES_PATH}/chute{number}'
        name = f'chute{number}-cam{c+1}-img'
        extension = 'jpg'
        extract_frames(file_path, name, extension, dest)
    index+=1

### 1.2 Removing delay frames

In [ ]:
safe_mkdir('./data/multicam_delay_frames')
chutes = listdir_nohidden_sorted(MULTICAM_FRAMES_PATH)

for chute in chutes:
    imgs = listdir_nohidden_sorted(chute)
    for img in tqdm(imgs):
        cam_number = int(img[cam_number_index])
        chute_number = int(img[chute_number_start: chute_number_end])
        img_number = int(img[img_number_start: img_number_end])
        camera = multicam_delay_frames[f'camera_{cam_number}']
        delay = camera[chute_number-1]
        source = img
        destination = './data/multicam_delay_frames' 
        if img_number <= delay:
            pass
            shutil.move(source, destination) 
            print(f'moving {source} to {destination}')

In [ ]:
print('Total delay frames to remove: ',multicam_delay_frames.iloc[:,1:].to_numpy().sum())
print('Total delay frames removed: ', len(listdir_nohidden_sorted(destination)))

### 1.3 Dividing frames in 10 folders (one for each class)

In [ ]:
multicam_labels.head(10)

In [15]:
multicam_classes = pd.read_excel(MULTICAM_EXCEL_PATH, sheet_name='classes', index_col=0)
multicam_classes.head(10)

,class
1,walking_or_standing_up
2,falling
3,lying_on_the_ground
4,crounching
5,moving_down
6,moving_up
7,sitting
8,lying_on_a_sofa
9,moving_horizontaly


In [17]:
chutes = listdir_nohidden_sorted(MULTICAM_FRAMES_PATH)
for c in multicam_classes['class']:
    safe_mkdir(f'{MULTICAM_FRAMES_CLASSES_PATH}/{c}')

In [108]:
classes = multicam_classes['class']
total_moved = 0
for chute in chutes:
    per_chute_moved = 0
    imgs = listdir_nohidden_sorted(chute)
    for img in (t:= tqdm(imgs)):
        chute_number = int(img[chute_number_start: chute_number_end])
        cam_number = int(img[cam_number_index])
        img_number = int(img[img_number_start:img_number_end])
        
        chute_rows = multicam_labels.loc[multicam_labels['scenario_number'] == chute_number]
        num_rows = chute_rows.shape[0]
        for row_number in range(num_rows):
            row = chute_rows.iloc[[row_number]]
            period_start = int(row['period_start'])
            period_end = int(row['period_end'])
            
            if period_start <= img_number <= period_end:
                class_number = int(row['class'])
                dest = f'{MULTICAM_FRAMES_CLASSES_PATH}/{classes[class_number]}'
                shutil.move(img, dest)
                per_chute_moved += 1
                total_moved += 1
                t.set_description(f'Working on chute {str(chute_number).zfill(2)}... Moved {per_chute_moved} images')
print(f'Total images moved: {total_moved}')

Working on chute 24... Moved 25208 images: 100%|█████████████████████████| 29355/29355 [01:12<00:00, 405.17it/s]

Total images moved: 144792


# 2. Split URFall in one folder per class

## 2.1 URFall Global Variables and Imports

In [33]:
URFALL_PATH = './data/URFall'
URFALL_FALLS_CAM0_FRAMES = f'{URFALL_PATH}/falls-cam0-rgb'
URFALL_FALLS_CAM0_LABELS = f'{URFALL_PATH}/urfall-cam0-falls.csv'
URFALL_CLASSES = {'-1':'not_lying', '0':'falling', '1':'lying' }

In [29]:
columns = ['sequence_name', 'frame_number', 'label', 'height_width_ratio', 'major_minor_ratio', 'bounding_box_occupancy','max_std_xz','hh_max_ratio','H','D','P40']
urfall_falls_cam0 = pd.read_csv(URFALL_FALLS_CAM0_LABELS, names=columns, usecols=['sequence_name', 'frame_number', 'label'] )
urfall_falls_cam0

,sequence_name,frame_number,label
0,fall-01,1,-1
1,fall-01,2,-1
2,fall-01,3,-1
3,fall-01,4,-1
4,fall-01,5,-1
...,...,...,...
2990,fall-30,66,1
2991,fall-30,67,1
2992,fall-30,68,1
2993,fall-30,69,1


## 2.2 Move URFall frames to one folder per class

In [46]:
urfall_frames = listdir_nohidden_sorted(URFALL_FALLS_CAM0_FRAMES)
for value in URFALL_CLASSES.values():
  safe_mkdir(f'{URFALL_PATH}/{value}')
  
for i, img in (t:=tqdm(enumerate(urfall_frames))):
  label = int(urfall_falls_cam0.iloc[i, -1:])
  key = str(label)
  dest = f'{URFALL_PATH}/{URFALL_CLASSES[key]}'
  t.set_description(f'Moving img n°: {i} to {URFALL_CLASSES[key]} because label {key}')
  shutil.move(img, dest)

Moving img n°: 2977 to falling because label 0: : 2967it [00:02, 1313.69it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

